In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_8e01ca4d30994b239a53c23104c881fd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ng6mKmGtGPTwjl1w-iDYC0hvQW0CDv_bM2qXoyl3oZRL',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_8e01ca4d30994b239a53c23104c881fd.get_object(Bucket='foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',Key='df_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_train = pd.read_csv(body)
df_train.head()


,Country,Alcoholic Beverages,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Meat,Milk - Excluding Butter,Pulses,Starchy Roots,Stimulants,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Confirmed
0,Albania,0.9120,16.0930,1.0591,16.2107,0.8091,0.1471,3.8688,9.9441,0.8091,1.2651,0.2501,3.4422,0.3972,33.9070,2.8244,2.7508,0.027817
1,Angola,1.9388,4.6927,0.2644,18.3521,0.0441,0.8372,2.9302,0.5067,1.4981,12.6239,0.0441,2.7539,0.0000,45.3184,4.2741,0.3525,0.000111
2,Antigua and Barbuda,2.3041,15.3672,1.5429,13.7215,0.2057,1.7280,7.0356,4.6904,0.4526,0.8434,0.4937,5.8218,0.0823,34.6225,4.6904,1.2960,0.025773
3,Argentina,1.4354,14.9869,1.0650,16.7927,0.8643,0.2006,9.4459,3.1641,0.1235,1.4045,0.2315,7.0536,0.0463,34.9900,5.5410,0.8643,0.010643
4,Armenia,0.2274,12.8330,1.7706,19.2658,0.7310,0.1787,4.2235,5.6368,0.4386,1.2508,0.6985,5.2956,0.3086,37.1670,3.5737,3.2164,0.080554


In [3]:

body = client_8e01ca4d30994b239a53c23104c881fd.get_object(Bucket='foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',Key='df_test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body)
df_test.head()


,Country,Alcoholic Beverages,Animal Products,Animal fats,Cereals - Excluding Beer,Eggs,"Fish, Seafood",Meat,Milk - Excluding Butter,Pulses,Starchy Roots,Stimulants,Sugar & Sweeteners,Treenuts,Vegetal Products,Vegetable Oils,Vegetables,Confirmed
0,Afghanistan,0.0000,4.7774,0.8504,37.1186,0.1501,0.0000,1.2006,2.4512,0.5003,0.3252,0.0750,2.2261,0.1251,45.2476,2.3012,0.7504,0.007108
1,Algeria,0.0896,6.0326,0.1941,25.0112,0.4181,0.1195,1.2543,3.9869,1.0900,1.9262,0.1493,3.9869,0.2240,43.9749,5.7638,2.0457,0.010307
2,Bangladesh,0.0000,2.3695,0.1541,37.5265,0.2312,0.8284,0.3468,0.7706,1.2329,1.9264,0.0193,1.4255,0.1156,47.6401,3.4290,0.6165,0.005777
3,Belize,1.5199,9.1752,2.6135,18.9064,0.2966,0.3707,3.9481,1.8906,2.2799,0.5005,0.2966,7.6367,0.1297,40.8526,5.1158,0.7414,0.004412
4,Brazil,1.4314,13.2061,1.5238,14.5913,0.4464,0.2309,7.1571,3.7402,2.0317,1.7854,0.1385,6.3260,0.0770,36.8016,7.2033,0.5541,0.048643


In [4]:
import numpy as np
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR

In [5]:
X_train = df_train[df_train.columns[1:17]] 
Y_train = df_train['Confirmed']
X_test = df_test[df_test.columns[1:17]] 
Y_test = df_test['Confirmed']
X_train_np=X_train.to_numpy()
Y_train_np=Y_train.to_numpy()
X_test_np=X_test.to_numpy()
Y_test_np=Y_test.to_numpy()

Lets define a SVR model and train with training set.

In [14]:
svr1=SVR(epsilon=0.001,kernel='rbf',gamma='scale',C=0.2)
svr1.fit(X_train_np, Y_train_np)

y_pred=svr1.predict(X_test_np)


As the ouput shuld not be negative,lets condition the predicted outputs.

In [15]:
for i in range(len(y_pred)):
    if(y_pred[i]<0):
        y_pred[i]=0

Consider "mean-squared error" as the criteria to evaluate model.

In [16]:
model_1_mse=mean_squared_error(Y_test_np, y_pred)
model_1_mse

0.001602298513105709

Now lets perform PCA to  reduce dimentionallity of fetures to 5.

In [9]:
from sklearn.decomposition import PCA

In [10]:
pca = PCA(n_components=5)
X_pca=pca.fit_transform(X_train_np)
X_test_pca=pca.transform(X_test_np)
X_pca.shape

(144, 5)

Lets train the SVR model with PCA features.

In [17]:
svr2=SVR(epsilon=0.001,kernel='rbf',gamma='scale',C=0.2)
svr2.fit(X_pca, Y_train_np)

y_pred_pca=svr2.predict(X_test_pca)

In [18]:
for i in range(len(y_pred_pca)):
    if(y_pred_pca[i]<0):
        y_pred_pca[i]=0

In [19]:
model_2_mse=mean_squared_error(Y_test_np, y_pred_pca)
model_2_mse

0.0010448858154379652

Here we can onserve that the mse reduced by about 37 % by using PCA for dimentionality reduction.

In [20]:
import pickle

In [21]:

pickle_out = open("model1.pickle","wb")
pickle.dump(svr1, pickle_out)
pickle_out.close()
pickle_out_2=open("model2.pickle","wb")
pickle.dump(svr2, pickle_out_2)
pickle_out_2.close()

In [22]:

# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-e8071002-7c9e-408a-9f8f-7095f09bcb51',
    'IBM_API_KEY_ID': 'ng6mKmGtGPTwjl1w-iDYC0hvQW0CDv_bM2qXoyl3oZRL',
    'ENDPOINT': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'BUCKET': 'foodimpactcovid-donotdelete-pr-ckuilhhenkqwsg',
    'FILE': 'Food_Supply_kcal_Data.csv'
}


In [23]:
client_8e01ca4d30994b239a53c23104c881fd.upload_file(Filename='model1.pickle',Bucket=credentials_1['BUCKET'],Key='model1.pickle')
client_8e01ca4d30994b239a53c23104c881fd.upload_file(Filename='model2.pickle',Bucket=credentials_1['BUCKET'],Key='model2.pickle')